In [1]:
import cv2 as cv
import numpy as np
import string
import random
import matplotlib.pyplot as plt
import ctypes 
import os
import pandas as pd
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten , Conv2D
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [79]:
# This function we use it to add new photos to DataSet folder
def new_user(name):
    
    def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
        return ''.join(random.choice(chars) for _ in range(size))

    def save_pic(event,x,y,flags,param):
         if event == cv.EVENT_RBUTTONDOWN:
                if not os.path.exists("DataSet\\"+name):
                    os.makedirs("DataSet\\"+name)
                image_name = "DataSet\\"+name+"\\"+id_generator()+".jpg"
                cv.imwrite(image_name,img_copy)
            
    cap = cv.VideoCapture(0)
    face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')

#     ctypes.windll.user32.MessageBoxW(0, "right click in the frame to take a picture", "worinning", 1)
    cv.namedWindow("frame")
    cv.setMouseCallback("frame",save_pic)
    while True:
        ret , frame = cap.read()
        gray_frame = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(frame, 1.1, 4)
        for (x, y, w, h) in faces:
            cv.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            img_copy = frame[y: y + h, x :x + w ]
        name = name
        cv.imshow("frame",frame)
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv.destroyAllWindows()

In [5]:
# new_user("Bahaa_with_mask")

In [2]:
# Part 2 data preprocessing import data and images augmentation

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        'DataSet',
        target_size=(256, 256),
        batch_size=32,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        'DataSet',
        target_size=(256, 256),
        batch_size=32,
        class_mode='binary')



Found 1801 images belonging to 2 classes.
Found 1801 images belonging to 2 classes.


In [17]:
# Part 1 Create model Classification

classifier = Sequential()


In [18]:
# Step 1 Convolution layer
classifier.add(Convolution2D(64,3,3,input_shape=(256, 256, 3),activation='relu')) 
classifier.add(Convolution2D(64,3,3,input_shape=(256, 256, 3),activation='relu')) 
classifier.add(Convolution2D(64,3,3,input_shape=(256, 256, 3),activation='relu')) 


C:\Users\Geek\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), input_shape=(256, 256,..., activation="relu")`
  
C:\Users\Geek\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), input_shape=(256, 256,..., activation="relu")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Geek\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), input_shape=(256, 256,..., activation="relu")`
  after removing the cwd from sys.path.


In [19]:
# Step 2 Max Pooling
classifier.add(MaxPool2D(pool_size=(2,2)))
classifier.add(MaxPool2D(pool_size=(2,2)))
classifier.add(MaxPool2D(pool_size=(2,2)))


In [20]:
# Add Images To flatten Layer 
classifier.add(Flatten())

In [21]:
# Add Images To ANN Layer 
classifier.add(Dense(output_dim=256,activation='relu'))
classifier.add(Dense(output_dim=128,activation='relu'))
classifier.add(Dense(output_dim=64,activation='relu'))
classifier.add(Dense(output_dim=1,activation='sigmoid'))


C:\Users\Geek\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=256)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Geek\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  after removing the cwd from sys.path.
C:\Users\Geek\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=64)`
  """
C:\Users\Geek\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1)`
  


In [22]:
# Compile the model
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [75]:
# train the model
classifier.fit_generator(
        training_set,
        steps_per_epoch=20,
        epochs=2,
        validation_data=test_set,
        validation_steps=10
)

Epoch 1/2
20/20 [==============================] - 294s 15s/step - loss: 0.1131 - acc: 0.9589 - val_loss: 0.0535 - val_acc: 0.9938
Epoch 2/2
20/20 [==============================] - 269s 13s/step - loss: 0.0378 - acc: 0.9922 - val_loss: 0.0453 - val_acc: 0.9938


In [78]:
# Test the model on real time camera
cap = cv.VideoCapture(0)
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
font = cv.FONT_HERSHEY_SIMPLEX 
  
# org 
org = (50, 50) 
  
# fontScale 
fontScale = 1
   
# Blue color in BGR 
color = (255, 0, 0) 
  
# Line thickness of 2 px 
thickness = 2
while True:
    ret , frame = cap.read()
    faces = face_cascade.detectMultiScale(frame, 1.1, 4)
    for (x, y, w, h) in faces:
        cv.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        img_copy = frame[y: y + h, x :x + w ]
        img_cc = cv.resize(img_copy,(256, 256,))
        image = np.expand_dims(img_cc, axis=0)
        y_pred = classifier.predict(image)
        if y_pred[0][0] == 1.0:
            cv.putText(frame, 'with Mask', org, font,fontScale, color, thickness, cv.LINE_AA)
        else:
            cv.putText(frame, 'Without Mask', org, font,fontScale, color, thickness, cv.LINE_AA)

    cv.imshow("frame",frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()